This year, Santa brought little Bobby Tables a set of wires and bitwise logic gates! Unfortunately, little Bobby is a little under the recommended age range, and he needs help assembling the circuit.

Each wire has an identifier (some lowercase letters) and can carry a 16-bit signal (a number from 0 to 65535). A signal is provided to each wire by a gate, another wire, or some specific value. Each wire can only get a signal from one source, but can provide its signal to multiple destinations. A gate provides no signal until all of its inputs have a signal.

The included instructions booklet describes how to connect the parts together: x AND y -> z means to connect wires x and y to an AND gate, and then connect its output to wire z.

For example:

123 -> x means that the signal 123 is provided to wire x.
x AND y -> z means that the bitwise AND of wire x and wire y is provided to wire z.
p LSHIFT 2 -> q means that the value from wire p is left-shifted by 2 and then provided to wire q.
NOT e -> f means that the bitwise complement of the value from wire e is provided to wire f.
Other possible gates include OR (bitwise OR) and RSHIFT (right-shift). If, for some reason, you'd like to emulate the circuit instead, almost all programming languages (for example, C, JavaScript, or Python) provide operators for these gates.

For example, here is a simple circuit:

123 -> x
456 -> y
x AND y -> d
x OR y -> e
x LSHIFT 2 -> f
y RSHIFT 2 -> g
NOT x -> h
NOT y -> i
After it is run, these are the signals on the wires:

d: 72
e: 507
f: 492
g: 114
h: 65412
i: 65079
x: 123
y: 456
In little Bobby's kit's instructions booklet (provided as your puzzle input), what signal is ultimately provided to wire a?

In [13]:
circuit_import = []

with open("Day7Input.txt", encoding="utf-8") as file:
    for line in file:
        circuit_import.append(line.strip())
        
circuit_instructions = []

for instruction in circuit_import:
    circuit_instructions.append(instruction.split())

print(circuit_instructions)

[['lf', 'AND', 'lq', '->', 'ls'], ['iu', 'RSHIFT', '1', '->', 'jn'], ['bo', 'OR', 'bu', '->', 'bv'], ['gj', 'RSHIFT', '1', '->', 'hc'], ['et', 'RSHIFT', '2', '->', 'eu'], ['bv', 'AND', 'bx', '->', 'by'], ['is', 'OR', 'it', '->', 'iu'], ['b', 'OR', 'n', '->', 'o'], ['gf', 'OR', 'ge', '->', 'gg'], ['NOT', 'kt', '->', 'ku'], ['ea', 'AND', 'eb', '->', 'ed'], ['kl', 'OR', 'kr', '->', 'ks'], ['hi', 'AND', 'hk', '->', 'hl'], ['au', 'AND', 'av', '->', 'ax'], ['lf', 'RSHIFT', '2', '->', 'lg'], ['dd', 'RSHIFT', '3', '->', 'df'], ['eu', 'AND', 'fa', '->', 'fc'], ['df', 'AND', 'dg', '->', 'di'], ['ip', 'LSHIFT', '15', '->', 'it'], ['NOT', 'el', '->', 'em'], ['et', 'OR', 'fe', '->', 'ff'], ['fj', 'LSHIFT', '15', '->', 'fn'], ['t', 'OR', 's', '->', 'u'], ['ly', 'OR', 'lz', '->', 'ma'], ['ko', 'AND', 'kq', '->', 'kr'], ['NOT', 'fx', '->', 'fy'], ['et', 'RSHIFT', '1', '->', 'fm'], ['eu', 'OR', 'fa', '->', 'fb'], ['dd', 'RSHIFT', '2', '->', 'de'], ['NOT', 'go', '->', 'gp'], ['kb', 'AND', 'kd', '->', 'k

In [14]:
def is_literal(value):
    return value.isnumeric()

In [15]:
def literal_pairs(value1, value2):
    return is_literal(value1) and is_literal(value2)

In [16]:
def shift_value(direction, value, amount):
    #True for right shift, false for left shift
    if direction:
        return value >> amount
    else:
        return value << amount

In [17]:
def one_part(instruction):
    if is_literal(instruction[0]):
        #If it's numeric, the value is sent straight to wire
        wires[instruction[-1]] = int(instruction[0])
        return True
    else:
        #If it's not numeric, check if wire exists first
        if instruction[0] in wires:
            #If it does, just use value from existing wire
            wires[instruction[-1]] = wires[instruction[0]]
            return True
        else:
            return False

In [18]:
def two_parts(instruction):
    if instruction[0] == "NOT":
        #If it's numeric, the bitewise complement of the value is found and sent to the wire
        if is_literal(instruction[1]):
            wires[instruction[-1]] = 65536 + ~int(instruction[1])
            return True
        #If it's not numeric
        else:
            #Check wire exists first, if it doesn't it's gotta be zeroed first
            if instruction[1] in wires:
                wires[instruction[-1]] = 65536 + ~wires[instruction[1]]
                return True
            else:
                return False
    else:
        print("Instruction not recognised, ignoring...")
        return False
    

In [19]:
def three_parts(instruction):
    if instruction[1] == "AND":
        if literal_pairs(instruction[0], instruction[2]):
            #Bitwise AND of 2 literals to the wire
            wires[instruction[-1]] = int(instruction[0]) & int(instruction[2])
            return True
        elif is_literal(instruction[0]):
            #Bitwise AND of a literal and a wire to the wire
            if instruction[2] in wires:
                wires[instruction[-1]] = int(instruction[0]) & wires[instruction[2]]
                return True
            return False
        elif is_literal(instruction[2]):
            #Bitwise AND of a wire and a literal to the wire
            if instruction[0] in wires:
                wires[instruction[-1]] = wires[instruction[0]] & int(instruction[2])
                return True
            return False
        else:
            if instruction[0] in wires and instruction[2] in wires:
                wires[instruction[-1]] = wires[instruction[0]] & wires[instruction[2]]
                return True
            return False
        
    elif instruction[1] == "OR":
        if literal_pairs(instruction[0], instruction[2]):
            #Bitwise OR of 2 literals to the wire
            wires[instruction[-1]] = int(instruction[0]) | int(instruction[2])
            return True
        elif is_literal(instruction[0]):
            #Bitwise OR of a literal and a wire to the wire
            if instruction[2] in wires:
                wires[instruction[-1]] = int(instruction[0]) | wires[instruction[2]]
                return True
            return False
        elif is_literal(instruction[2]):
            #Bitwise OR of a wire and a literal to the wire
            if instruction[0] in wires:
                wires[instruction[-1]] = wires[instruction[0]] | int(instruction[2])
                return True
            return False
        else:
            if instruction[0] in wires and instruction[2] in wires:
                wires[instruction[-1]] = wires[instruction[0]] | wires[instruction[2]]
                return True
            return False
        
    elif instruction[1] == "LSHIFT":
        #Check if we need to left shift a literal or a wire
        if is_literal(instruction[0]):
            #Left shifting a value
            wires[instruction[-1]] = shift_value(False, int(instruction[0]), int(instruction[2]))
            return True
        else:
            #Left shifting a wire
            if instruction[0] in wires:
                wires[instruction[-1]] = shift_value(False, wires[instruction[0]], int(instruction[2]))
                return True
            return False
        
    elif instruction[1] == "RSHIFT":
        #Check if we need to right shift a literal or a wire
        if is_literal(instruction[0]):
            #Right shifting a value
            wires[instruction[-1]] = shift_value(True, int(instruction[0]), int(instruction[2]))
            return True
        else:
            #Right shifting a wire
            if instruction[0] in wires:
                wires[instruction[-1]] = shift_value(True, wires[instruction[0]], int(instruction[2]))
                return True
            return False
    else:
        print("Instruction not recognised, ignoring...")

In [20]:
def find_remaining(instructions):
    remaining_instructions = []
    for instruction in instructions:
        if len(instruction) == 3:
            if not one_part(instruction):
                remaining_instructions.append(instruction)
        elif len(instruction) == 4:
            if not two_parts(instruction):
                remaining_instructions.append(instruction)
        elif len(instruction) == 5:
            if not three_parts(instruction):
                remaining_instructions.append(instruction)
        else:
            print("Instruction not recognised, ignoring...")
    print(f"Wires: {wires}\n")
    print(f"Remaining instructions: {remaining_instructions}\n")
    return remaining_instructions

In [21]:
wires = {}

while len(circuit_instructions) > 0:
    circuit_instructions = find_remaining(circuit_instructions)
        
print(wires)


Wires: {'b': 19138, 'c': 0, 't': 0, 'v': 9569, 'f': 598, 'd': 4784, 'e': 2392, 'h': 80, 'i': 65455}

Remaining instructions: [['lf', 'AND', 'lq', '->', 'ls'], ['iu', 'RSHIFT', '1', '->', 'jn'], ['bo', 'OR', 'bu', '->', 'bv'], ['gj', 'RSHIFT', '1', '->', 'hc'], ['et', 'RSHIFT', '2', '->', 'eu'], ['bv', 'AND', 'bx', '->', 'by'], ['is', 'OR', 'it', '->', 'iu'], ['b', 'OR', 'n', '->', 'o'], ['gf', 'OR', 'ge', '->', 'gg'], ['NOT', 'kt', '->', 'ku'], ['ea', 'AND', 'eb', '->', 'ed'], ['kl', 'OR', 'kr', '->', 'ks'], ['hi', 'AND', 'hk', '->', 'hl'], ['au', 'AND', 'av', '->', 'ax'], ['lf', 'RSHIFT', '2', '->', 'lg'], ['dd', 'RSHIFT', '3', '->', 'df'], ['eu', 'AND', 'fa', '->', 'fc'], ['df', 'AND', 'dg', '->', 'di'], ['ip', 'LSHIFT', '15', '->', 'it'], ['NOT', 'el', '->', 'em'], ['et', 'OR', 'fe', '->', 'ff'], ['fj', 'LSHIFT', '15', '->', 'fn'], ['t', 'OR', 's', '->', 'u'], ['ly', 'OR', 'lz', '->', 'ma'], ['ko', 'AND', 'kq', '->', 'kr'], ['NOT', 'fx', '->', 'fy'], ['et', 'RSHIFT', '1', '->', 'fm'

In [22]:
print(wires["a"])

16076


In [23]:
def find_remaining_after_reset(instructions):
    remaining_instructions = []
    for instruction in instructions:
        if instruction[-1] != "b":
            if len(instruction) == 3:
                if not one_part(instruction):
                    remaining_instructions.append(instruction)
            elif len(instruction) == 4:
                if not two_parts(instruction):
                    remaining_instructions.append(instruction)
            elif len(instruction) == 5:
                if not three_parts(instruction):
                    remaining_instructions.append(instruction)
            else:
                print("Instruction not recognised, ignoring...")
        else:
            print("This would rewrite b, ignoring...")
    print(f"Wires: {wires}\n")
    print(f"Remaining instructions: {remaining_instructions}\n")
    return remaining_instructions

In [24]:
signal = wires["a"]
wires = {"b": signal}
print(wires)

circuit_instructions = []

for instruction in circuit_import:
    circuit_instructions.append(instruction.split())

while len(circuit_instructions) > 0:
    circuit_instructions = find_remaining_after_reset(circuit_instructions)

print(wires)
print(wires["a"])

{'b': 16076}
This would rewrite b, ignoring...
Wires: {'b': 16076, 'c': 0, 't': 0, 'v': 8038, 'f': 502, 'd': 4019, 'e': 2009, 'h': 464, 'i': 65071}

Remaining instructions: [['lf', 'AND', 'lq', '->', 'ls'], ['iu', 'RSHIFT', '1', '->', 'jn'], ['bo', 'OR', 'bu', '->', 'bv'], ['gj', 'RSHIFT', '1', '->', 'hc'], ['et', 'RSHIFT', '2', '->', 'eu'], ['bv', 'AND', 'bx', '->', 'by'], ['is', 'OR', 'it', '->', 'iu'], ['b', 'OR', 'n', '->', 'o'], ['gf', 'OR', 'ge', '->', 'gg'], ['NOT', 'kt', '->', 'ku'], ['ea', 'AND', 'eb', '->', 'ed'], ['kl', 'OR', 'kr', '->', 'ks'], ['hi', 'AND', 'hk', '->', 'hl'], ['au', 'AND', 'av', '->', 'ax'], ['lf', 'RSHIFT', '2', '->', 'lg'], ['dd', 'RSHIFT', '3', '->', 'df'], ['eu', 'AND', 'fa', '->', 'fc'], ['df', 'AND', 'dg', '->', 'di'], ['ip', 'LSHIFT', '15', '->', 'it'], ['NOT', 'el', '->', 'em'], ['et', 'OR', 'fe', '->', 'ff'], ['fj', 'LSHIFT', '15', '->', 'fn'], ['t', 'OR', 's', '->', 'u'], ['ly', 'OR', 'lz', '->', 'ma'], ['ko', 'AND', 'kq', '->', 'kr'], ['NOT', 'fx